In [1]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [52]:
train = pd.read_csv(r'C:\Users\gmlkd\machine-learning-exercises\10.DAYCON대회_사용자불편예지\training set\train_concat.csv')
test = pd.read_csv(r'C:\Users\gmlkd\machine-learning-exercises\10.DAYCON대회_사용자불편예지\training set\test_concat.csv')
train_problem = pd.read_csv(r'C:\Users\gmlkd\machine-learning-exercises\10.DAYCON대회_사용자불편예지\preprocessed data\new_train_problem.csv')
submission = pd.read_csv("C:/Users/gmlkd/data/시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회_data/sample_submission.csv")

In [3]:
train = train.to_numpy()
problem = np.zeros(15000)
problem[train_problem.user_id.unique() - 10000] = 1

train_x = train
train_y = problem

In [4]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):
    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 1000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 3
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

[LightGBM] [Info] Number of positive: 3633, number of negative: 8367
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002501 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4699
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 47
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.302750 -> initscore=-0.834237
[LightGBM] [Info] Start training from score -0.834237
Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[2]	valid_0's auc: 0.793716	valid_0's pr_auc: 0.798649
[LightGBM] [Info] Number of positive: 4828, number of negative: 7172
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001974 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4860
[LightGBM] [Info] Number of data points in the train set: 12000, number of used feature

In [5]:
print(np.mean(auc_scores))

0.7981357833917746


In [6]:
train_x = pd.DataFrame(train_x)
train_y = pd.DataFrame(train_y)
train_y.columns = ['problem']
train = pd.concat([train_x,train_y], axis =1)
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,problem
0,-0.213614,-0.306457,-0.233452,-0.241292,0.073584,-0.056498,-0.049284,-0.02228,-0.051123,-0.029449,-0.005874,-0.01187,-0.007933,-0.008729,-0.100061,-0.006074,-0.010215,-0.033592,-0.271696,-0.008946,-0.012999,-0.012935,-0.017218,-0.010045,-0.007109,-0.00628,-0.013052,-0.006892,-0.024113,-0.011174,-0.196237,-0.030361,0.11631,-0.268954,-0.075856,-0.048805,-0.042845,0.012048,0.025702,-0.025070,-0.658566,-0.155069,-0.212251,-0.092773,-0.212920,-0.218524,-0.076541,-0.088543,-0.146097,-0.153975,-0.182051,-0.199422,-1.025389,-0.056732,-0.074532,-1.017440,-0.123447,-0.083385,-0.602526,-0.361839,-0.644791,0.443964,0.0,0.0
1,0.393497,-0.306457,-0.233452,2.287379,-0.045112,-0.056498,-0.049284,-0.02228,-0.051123,-0.029449,-0.005874,-0.01187,-0.007933,-0.008729,-0.100061,-0.006074,-0.010215,-0.033592,-0.271696,-0.008946,-0.012999,-0.012935,-0.017218,-0.010045,-0.007109,-0.00628,-0.013052,-0.006892,-0.024113,-0.011174,-0.196237,-0.030361,-0.03337,0.045429,-0.075856,-0.048805,-0.042845,-0.028081,0.025702,-0.031916,0.558991,-0.155069,4.393257,1.755177,4.247391,1.098295,0.064463,1.052892,-0.586751,-0.153975,0.152699,-0.199422,-0.162199,0.076751,-0.074532,0.610876,-0.123447,-0.083385,0.798137,2.634011,-0.362708,0.443964,0.0,1.0
2,-0.216905,-0.306457,-0.233452,-0.241292,0.069452,-0.056498,-0.049284,-0.02228,-0.051123,-0.029449,-0.005874,-0.01187,-0.007933,-0.008729,-0.100061,-0.006074,-0.010215,-0.033592,-0.271696,-0.008946,-0.012999,-0.012935,-0.017218,-0.010045,-0.007109,-0.00628,-0.013052,-0.006892,-0.024113,-0.011174,-0.196237,-0.030361,0.00405,-0.262404,-0.046226,-0.048805,-0.042845,-0.022348,0.025702,-0.045610,-0.760029,-0.155069,-0.212251,-0.092773,-0.212920,-0.218524,-0.020139,-0.088543,-0.454555,-0.153975,-0.182051,-0.199422,-1.025389,-0.056732,-0.074532,-1.017440,-0.123447,-0.083385,-0.602526,-0.361839,-0.644791,-0.420972,0.0,0.0
3,-0.219299,-0.306457,-0.233452,0.078970,-0.045112,-0.056498,-0.049284,-0.02228,-0.051123,-0.029449,-0.005874,-0.01187,-0.007933,-0.008729,-0.100061,-0.006074,-0.010215,-0.033592,-0.271696,-0.008946,-0.012999,-0.012935,-0.017218,-0.010045,-0.007109,-0.00628,-0.013052,-0.006892,-0.024113,-0.011174,-0.196237,-0.030361,-0.03337,-0.255854,-0.075856,-0.048805,-0.042845,-0.028081,-0.079829,-0.059303,-0.861492,-0.105799,-0.212251,-0.092773,-0.212920,-0.169753,-0.076541,-0.088543,0.955538,-0.153975,-0.095016,-0.199422,-0.334837,-0.056732,-0.074532,0.610876,0.673531,-0.083385,-0.391807,-0.308342,-0.644791,0.443964,0.0,0.0
4,-0.075675,0.569171,-0.233452,-0.241292,-0.045112,-0.056498,-0.049284,-0.02228,-0.051123,-0.029449,-0.005874,-0.01187,-0.007933,-0.008729,-0.100061,-0.006074,-0.010215,-0.033592,0.148885,-0.008946,-0.012999,-0.012935,-0.017218,-0.010045,-0.007109,-0.00628,-0.013052,-0.006892,-0.024113,-0.011174,-0.196237,-0.030361,-0.03337,-0.268954,-0.016597,-0.048805,-0.042845,-0.028081,0.236765,-0.042186,0.558991,-0.155069,-0.212251,-0.092773,-0.212920,0.025331,-0.076541,-0.088543,-0.102031,-0.153975,0.053613,-0.199422,0.355716,-0.056732,-0.074532,0.610876,-0.123447,-0.083385,-0.552945,-0.361839,-0.080625,0.443964,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,-0.251016,-0.306457,-0.233452,-0.035332,-0.045112,-0.056498,-0.049284,-0.02228,-0.051123,-0.029449,-0.005874,-0.01187,-0.007933,-0.008729,-0.100061,-0.006074,-0.010215,-0.033592,-0.271696,-0.008946,-0.012999,-0.012935,-0.017218,-0.010045,-0.007109,-0.00628,-0.013052,-0.006892,-0.024113,-0.011174,-0.196237,-0.030361,-0.03337,-0.255854,0.042663,-0.048805,-0.042845,-0.028081,0.447828,-0.031916,-1.136892,-0.155069,-0.212251,-0.092773,-0.212920,-0.188913,-

In [7]:
clf = setup(data = train, target = "problem", silent=True) 

,Description,Value
0,session_id,1803
1,Target,problem
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 64)"
5,Missing Values,True
6,Numeric Features,59
7,Categorical Features,4
8,Ordinal Features,False
9,High Cardinality Features,False


In [8]:
best_3 = compare_models(sort = 'AUC', n_select = 3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7909,0.8075,0.4779,0.8155,0.6025,0.4735,0.5047,0.2700
lightgbm,Light Gradient Boosting Machine,0.7891,0.8046,0.5109,0.7774,0.6164,0.4794,0.4996,0.0470
rf,Random Forest Classifier,0.7880,0.8035,0.4885,0.7927,0.6044,0.4705,0.4963,0.1620
et,Extra Trees Classifier,0.7816,0.7964,0.4759,0.7800,0.5909,0.4537,0.4795,0.1680
ada,Ada Boost Classifier,0.7835,0.7940,0.4842,0.7795,0.5972,0.4601,0.4845,0.0770
lr,Logistic Regression,0.7766,0.7696,0.4253,0.8114,0.5579,0.4273,0.4673,0.3440
lda,Linear Discriminant Analysis,0.7625,0.7506,0.3564,0.8313,0.4986,0.3741,0.4319,0.0310
nb,Naive Bayes,0.6985,0.7342,0.5060,0.6250,0.5271,0.3223,0.3395,0.0160
knn,K Neighbors Classifier,0.7455,0.7207,0.4276,0.6874,0.5270,0.3656,0.3850,0.3690
dt,Decision Tree Classifier,0.6956,0.6588,0.5505,0.5404,0.5452,0.3166,0.3167,0.0300


In [9]:
blended = blend_models(estimator_list = best_3, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7852,0.7864,0.4871,0.7829,0.6005,0.4644,0.4888
1,0.7919,0.8137,0.4914,0.8047,0.6102,0.4793,0.5064
2,0.7900,0.8124,0.4978,0.7922,0.6115,0.4776,0.5018
3,0.7948,0.8263,0.4821,0.8276,0.6092,0.4829,0.5153
4,0.7928,0.8134,0.4828,0.8175,0.6070,0.4787,0.5093
Mean,0.7909,0.8104,0.4882,0.8050,0.6077,0.4766,0.5044
Std,0.0032,0.0131,0.0059,0.0162,0.0039,0.0064,0.0089


In [10]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7916,0.8196,0.4921,0.8173,0.6143,0.4833,0.5125


In [11]:
final_model = finalize_model(blended)

In [13]:
# pycaret의 predict_model을 사용하기 위해 test의 모든 컬럼을 정수로 변경
t_columns = list(np.arange(0,63))
test.columns = t_columns

In [14]:
prep_pipe = get_config('prep_pipe')
transformed_unseen_data = prep_pipe.transform(test)

In [54]:
prections = final_model.predict_proba(transformed_unseen_data)[:,1]
prections

array([0.93231924, 0.18310495, 0.29991395, ..., 0.5051749 , 0.8358318 ,
       0.37914147])

In [70]:
submission.drop(index=13262, inplace=True)

In [71]:
submission['problem'] = prections

In [55]:
prections.mean()

0.33475522947947833

In [72]:
user_43262 = pd.DataFrame({"user_id":[43262], "problem":[0.33]})
user_43262

,user_id,problem
0,43262,0.33


In [73]:
# 예측 결과값의 평균으로 넣어준다.
sub = pd.concat([submission[:13262], user_43262, submission[13262:]], ignore_index=True)
sub.iloc[13262, :]

user_id    43262.00
problem        0.33
Name: 13262, dtype: float64

In [74]:
sub.user_id = sub.user_id.astype(int)

In [75]:
sub

,user_id,problem
0,30000,0.932319
1,30001,0.183105
2,30002,0.299914
3,30003,0.765894
4,30004,0.886952
...,...,...
14994,44994,0.285616
14995,44995,0.369023
14996,44996,0.505175
14997,44997,0.835832


In [76]:
sub.to_csv('../submissions/submission_concat.csv', index=False)